In [1]:
import tensorflow as tf
tf.__version__

'2.2.0'

In [ ]:
from collections import Counter
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
import pandas as pd

magazine = pd.read_json("/content/drive/My Drive/branch/data/magazine.json", lines = True)
magazine.head()

,magazine_tag_list,id
0,"[브런치북, 육아일기, 대화법, 들려주고픈이야기]",38842
1,"[tea, food]",11540
2,[food],11541
3,"[브런치북, 일상, 시, 사람]",11546
4,"[감성에세이, 노래, 음악에세이]",11544


magazine.shape
(27967, 2)

In [ ]:
magazine_cnt = magazine['id'].value_counts()
magazine_cnt

2047     1
23969    1
42366    1
30084    1
28039    1
        ..
19196    1
23294    1
21247    1
43776    1
0        1
Name: id, Length: 27967, dtype: int64

In [ ]:
type(magazine_cnt) # pandas.core.series.Series
len(magazine_cnt.unique()) # 1
len(magazine['id'].unique()) # 27967

27967

In [ ]:
magazine.to_csv('/content/drive/My Drive/branch/data/magazine.csv', index = False)

In [ ]:
magazine_df = pd.read_csv('/content/drive/My Drive/branch/data/magazine.csv', index_col='id')
magazine_df.head()

,magazine_tag_list
id,
38842,"['브런치북', '육아일기', '대화법', '들려주고픈이야기']"
11540,"['tea', 'food']"
11541,['food']
11546,"['브런치북', '일상', '시', '사람']"
11544,"['감성에세이', '노래', '음악에세이']"


In [ ]:
metadata = pd.read_json("/content/drive/My Drive/branch/data/metadata.json", lines=True)
metadata.head()

,magazine_id,user_id,title,keyword_list,display_url,sub_title,reg_ts,article_id,id
0,8982,@bookdb,"사진으로 옮기기에도 아까운, 리치필드 국립공원","[여행, 호주, 국립공원]",https://brunch.co.kr/@bookdb/782,세상 어디에도 없는 호주 Top 10,1474944427000,782,@bookdb_782
1,12081,@kohwang56,[시] 서러운 봄,"[목련꽃, 아지랑이, 동행]",https://brunch.co.kr/@kohwang56/81,,1463092749000,81,@kohwang56_81
2,0,@hannahajink,무엇을 위해,[],https://brunch.co.kr/@hannahajink/4,무엇 때문에,1447997287000,4,@hannahajink_4
3,16315,@bryceandjuli,싫다,"[감정, 마음, 위로]",https://brunch.co.kr/@bryceandjuli/88,,1491055161000,88,@bryceandjuli_88
4,29363,@mijeongpark,Dubliner#7,"[유럽여행, 더블린, 아일랜드]",https://brunch.co.kr/@mijeongpark/34,#7. 내 친구의 집은 어디인가,1523292942000,34,@mijeongpark_34


In [ ]:
metadata.shape

(643104, 9)

In [ ]:
len(metadata['magazine_id'].unique())

28028

In [ ]:
metadata_cnt= metadata['magazine_id'].value_counts()
len(metadata_cnt.unique()) #318
metadata_cnt.unique()

array([160823,   2430,   1417,   1300,   1279,   1069,    988,    889,
          816,    784,    692,    664,    622,    617,    611,    607,
          551,    523,    504,    498,    492,    488,    486,    481,
          479,    478,    469,    465,    459,    458,    454,    444,
          438,    429,    420,    415,    413,    409,    407,    406,
          403,    390,    385,    380,    376,    374,    372,    371,
          370,    369,    366,    365,    364,    363,    362,    360,
          354,    353,    350,    348,    340,    337,    336,    335,
          334,    330,    328,    327,    322,    321,    319,    311,
          307,    306,    305,    295,    290,    282,    280,    277,
          273,    270,    269,    266,    265,    261,    259,    256,
          254,    252,    251,    250,    249,    246,    245,    243,
          242,    241,    240,    238,    237,    236,    233,    231,
          230,    229,    228,    226,    225,    224,    221,    220,
      

magazine id는 중복되지 않음.
metadata의 magazine_id 는 중복 된 것이 많음.
즉, magazine id 별 metadata는 여러개임.

In [ ]:
metadata.to_csv('/content/drive/My Drive/branch/data/metadata.csv', index = False)

In [ ]:
 metadata_df= pd.read_csv('/content/drive/My Drive/branch/data/metadata.csv', index_col='magazine_id')
metadata_df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,user_id,title,keyword_list,display_url,sub_title,reg_ts,article_id,id
magazine_id,,,,,,,,
8982.0,@bookdb,"사진으로 옮기기에도 아까운, 리치필드 국립공원","['여행', '호주', '국립공원']",https://brunch.co.kr/@bookdb/782,세상 어디에도 없는 호주 Top 10,1.47494e+12,782.0,@bookdb_782
12081.0,@kohwang56,[시] 서러운 봄,"['목련꽃', '아지랑이', '동행']",https://brunch.co.kr/@kohwang56/81,NaN,1.46309e+12,81.0,@kohwang56_81
0.0,@hannahajink,무엇을 위해,[],https://brunch.co.kr/@hannahajink/4,무엇 때문에,1.448e+12,4.0,@hannahajink_4
16315.0,@bryceandjuli,싫다,"['감정', '마음', '위로']",https://brunch.co.kr/@bryceandjuli/88,NaN,1.49106e+12,88.0,@bryceandjuli_88
29363.0,@mijeongpark,Dubliner#7,"['유럽여행', '더블린', '아일랜드']",https://brunch.co.kr/@mijeongpark/34,#7. 내 친구의 집은 어디인가,1.52329e+12,34.0,@mijeongpark_34


In [ ]:
# magazine_join = pd.concat([magazine, metadata], join='outer', axis=1)
magazine_join = pd.merge(magazine_df, metadata_df, how='outer', left_index=True, right_index=True)
magazine_join.head()

,magazine_tag_list,user_id,title,keyword_list,display_url,sub_title,reg_ts,article_id,id
0.0,[],@hannahajink,무엇을 위해,[],https://brunch.co.kr/@hannahajink/4,무엇 때문에,1.448e+12,4.0,@hannahajink_4
0.0,[],@purnbd,동시빵가게,"['도시애벌레', '공부', '동시']",https://brunch.co.kr/@purnbd/9,8. 동시빵 맛보기-'도시 애벌레',1.50569e+12,9.0,@purnbd_9
0.0,[],@purnbd,동시빵가게,"['주머니', '동시', '빵가게']",https://brunch.co.kr/@purnbd/7,6. 동시빵 맛보기- '펠리컨',1.50448e+12,7.0,@purnbd_7
0.0,[],@fskene,첫사랑 이였을까?,"['감성에세이', '사랑', '첫사랑']",https://brunch.co.kr/@fskene/247,NaN,1.47794e+12,247.0,@fskene_247
0.0,[],@hyunj2407,기억을 쌓아 상영하는 곳,"['문화극장', '극장']",https://brunch.co.kr/@hyunj2407/14,2016.09. 동두천 문화극장,1.47411e+12,14.0,@hyunj2407_14


In [ ]:
magazine_join.shape

(643212, 9)

In [ ]:
magazine_join.head(30)

,magazine_tag_list,user_id,title,keyword_list,display_url,sub_title,reg_ts,article_id,id
0.0,[],@hannahajink,무엇을 위해,[],https://brunch.co.kr/@hannahajink/4,무엇 때문에,1.448e+12,4.0,@hannahajink_4
0.0,[],@purnbd,동시빵가게,"['도시애벌레', '공부', '동시']",https://brunch.co.kr/@purnbd/9,8. 동시빵 맛보기-'도시 애벌레',1.50569e+12,9.0,@purnbd_9
0.0,[],@purnbd,동시빵가게,"['주머니', '동시', '빵가게']",https://brunch.co.kr/@purnbd/7,6. 동시빵 맛보기- '펠리컨',1.50448e+12,7.0,@purnbd_7
0.0,[],@fskene,첫사랑 이였을까?,"['감성에세이', '사랑', '첫사랑']",https://brunch.co.kr/@fskene/247,NaN,1.47794e+12,247.0,@fskene_247
0.0,[],@hyunj2407,기억을 쌓아 상영하는 곳,"['문화극장', '극장']",https://brunch.co.kr/@hyunj2407/14,2016.09. 동두천 문화극장,1.47411e+12,14.0,@hyunj2407_14
0.0,[],@ubermensch,17년 만에 다시 본 매트릭스,"['영화', '매트릭스', '리뷰']",https://brunch.co.kr/@ubermensch/24,세번 봐도 괜찮을 철학적 SF 액션 영화,1.4772e+12,24.0,@ubermensch_24
0.0,[],@thsgmlgns2,해설자 S 가 말하는 Z의 각론#3,"['살구', '어깨', '해설자']",https://brunch.co.kr/@thsgmlgns2/16,S의 독백,1.48018e+12,16.0,@thsgmlgns2_16
0.0,[],@bluehighway,캠프 엑스레이,"['캠프', '엑스레이']",https://brunch.co.kr/@bluehighway/34,미국에서도 우리나라에서도 꼭 봐야 하는 영화,1.48025e+12,34.0,@bluehighway_34
0.0,[],@purnbd,동시빵가게,"['동시', '빵가게']",https://brunch.co.kr/@purnbd/3,3. 동시 빵가게 오픈 임박~!,1.50268e+12,3.0,@purnbd_3
0.0,[],@junyoungcho,"(1)민간주도 스마트 시티, 볼리비아 산타크루즈 신도시","['스마트시티', '볼리비아', '산타크루즈']",https://brunch.co.kr/@junyoungcho/2,신흥국의 스마트 시티를 가다.,1.49839e+12,2.0,@junyoungcho_2


In [ ]:
import pandas as pd

users = pd.read_json('/content/drive/My Drive/branch/data/users.json', lines=True)
users.shape

(310758, 3)

In [ ]:
users.head()

,keyword_list,following_list,id
0,[],"[@perytail, @brunch]",#901985d8bc4c481805c4a4f911814c4a
1,[],"[@holidaymemories, @wadiz, @sciforus, @dailydu...",#1fd89e9dcfa64b45020d9eaca54e0eed
2,[],"[@commerceguy, @sunsutu, @kakao-it, @joohoonja...",#1d94baaea71a831e1f33e1c6bd126ed5
3,[],"[@amberjeon48, @forsy20, @nemotokki, @hawann, ...",#04641c01892b12dc018b1410e4928c0d
4,[],"[@dwcha7342, @iammento, @kakao-it, @dkam, @ant...",#65bcaff862aadff877e461f54187ab62


In [ ]:
users

,keyword_list,following_list,id
0,[],"[@perytail, @brunch]",#901985d8bc4c481805c4a4f911814c4a
1,[],"[@holidaymemories, @wadiz, @sciforus, @dailydu...",#1fd89e9dcfa64b45020d9eaca54e0eed
2,[],"[@commerceguy, @sunsutu, @kakao-it, @joohoonja...",#1d94baaea71a831e1f33e1c6bd126ed5
3,[],"[@amberjeon48, @forsy20, @nemotokki, @hawann, ...",#04641c01892b12dc018b1410e4928c0d
4,[],"[@dwcha7342, @iammento, @kakao-it, @dkam, @ant...",#65bcaff862aadff877e461f54187ab62
...,...,...,...
310753,[],"[@login002, @kkonal, @leeraha, @tobeme, @sohyu...",#2863e47d50f1640df6dac10b7bad94fb
310754,[],"[@simplelife-1p, @mint5051, @thecapitalist, @s...",#4341a155d1966e5618e310c45386aea4
310755,[],"[@cometseeker, @bijou, @suhanjang, @brunch]",#0d70f397a78d2ef638f812592fa8e6ba
310756,"[{'cnt': 1, 'keyword': '꽃병 꽃꽂이'}, {'cnt': 1, '...","[@taekangk, @cielbleu, @yongisa, @joongheekim,...",#1bbf5e3d1e4c373103981cdd819812da


In [ ]:
import glob
read_file_lst = glob.glob('/content/drive/My Drive/branch/read/read/*')

In [ ]:
exclude_file_lst = ['read.tar']

In [ ]:
read_df_lst = []
for f in read_file_lst:
    file_name = os.path.basename(f)
    if file_name in exclude_file_lst:
        print(file_name)
    else:
        df_temp = pd.read_csv(f, header=None, names=['raw'])
        df_temp['dt'] = file_name[:8]
        df_temp['hr'] = file_name[8:10]
        df_temp['user_id'] = df_temp['raw'].str.split(' ').str[0]
        df_temp['article_id'] = df_temp['raw'].str.split(' ').str[1:].str.join(' ').str.strip()
        read_df_lst.append(df_temp)

In [ ]:
read = pd.concat(read_df_lst)

In [ ]:
read.shape

(3507097, 5)

In [ ]:
read.head()

,raw,dt,hr,user_id,article_id
0,#3d6d98f06ae6024da6e01af11f19dfcb @seongheelee...,20190122,19,#3d6d98f06ae6024da6e01af11f19dfcb,@seongheeleelrwn_2 @thewatermelon_14 @thewater...
1,#a7f157a8a3fa2cb934799b5943dc34a0 @brunch_133,20190122,19,#a7f157a8a3fa2cb934799b5943dc34a0,@brunch_133
2,#40a04d5f58e3eb86f249f099a1580b2c @haereka_93,20190122,19,#40a04d5f58e3eb86f249f099a1580b2c,@haereka_93
3,#65a8fb00713e0b5bb30f129794e49cfa @shala_7 @ca...,20190122,19,#65a8fb00713e0b5bb30f129794e49cfa,@shala_7 @cardnews_10
4,#cd221250949c3f97988d43d68decb744 @bang1999_46...,20190122,19,#cd221250949c3f97988d43d68decb744,@bang1999_467 @bang1999_467 @bang1999_468


In [ ]:
col = ['dt', 'hr', 'user_id', 'article_id']
read_df = read[col]
read_df.head()

,dt,hr,user_id,article_id
0,20190122,19,#3d6d98f06ae6024da6e01af11f19dfcb,@seongheeleelrwn_2 @thewatermelon_14 @thewater...
1,20190122,19,#a7f157a8a3fa2cb934799b5943dc34a0,@brunch_133
2,20190122,19,#40a04d5f58e3eb86f249f099a1580b2c,@haereka_93
3,20190122,19,#65a8fb00713e0b5bb30f129794e49cfa,@shala_7 @cardnews_10
4,20190122,19,#cd221250949c3f97988d43d68decb744,@bang1999_467 @bang1999_467 @bang1999_468


In [ ]:
read_df.to_csv('/content/drive/My Drive/branch/data/read.csv', index = False)

In [ ]:
df_read= pd.read_csv('/content/drive/My Drive/branch/data/read.csv')
df_read.head()

,dt,hr,user_id,article_id
0,20190122,19,#3d6d98f06ae6024da6e01af11f19dfcb,@seongheeleelrwn_2 @thewatermelon_14 @thewater...
1,20190122,19,#a7f157a8a3fa2cb934799b5943dc34a0,@brunch_133
2,20190122,19,#40a04d5f58e3eb86f249f099a1580b2c,@haereka_93
3,20190122,19,#65a8fb00713e0b5bb30f129794e49cfa,@shala_7 @cardnews_10
4,20190122,19,#cd221250949c3f97988d43d68decb744,@bang1999_467 @bang1999_467 @bang1999_468


In [ ]:
df_read.shape

(3507097, 4)